In [1]:
from garden_ai import GardenClient, Model, step
from garden_ai.mlmodel import LocalModel

In [2]:
# instantiate the Garden client
client = GardenClient()

In [3]:
# convert our saved model into a Garden LocalModel
with open("requirements.txt", "r") as f:
    pip_reqs = [line.strip() for line in f.readlines()]

local_model = LocalModel(model_name="dendrite_segmentation",
                         flavor="tensorflow",
                         extra_pip_requirements=pip_reqs,
                         local_path="model.h5",
                         user_email=client.get_email())

In [ ]:
# register the LocalModel with Garden using the client
registered_model = client.register_model(local_model)

In [ ]:
# Garden's pipelines are composed of steps, so let's make one to run our model
@step
def run_inference(input_arg: object, model=Model(registered_model.model_name)) -> object:
    return model.predict(input_arg)

In [ ]:
# take the step we made and add it to a pipeline
pipeline = client.create_pipeline(client.get_email(),  # is this a real name?
                                  "Dendrite Segmentation Test Pipeline",
                                  short_name=registered_model.model_name,
                                  steps=(run_inference,),
                                  requirements_file="requirements.txt",
                                  description="Semantic dendrite segmentation via machine learning",
                                  version="1.0.0",
                                  tags=["materials science",
                                        "x-ray",
                                        "segmentation",
                                        "computer vision"])

In [ ]:
# build a container designed specially to run our pipeline
container_uuid = client.build_container(pipeline)

In [ ]:
# register the pipeline and its encompassing container to Garden!
func_uuid = client.register_pipeline(pipeline, container_uuid)
print(f"Created function with uuid: {func_uuid}!")